Explorando dados

In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd 

navegador = webdriver.Chrome()
navegador.get('https://www.iso.org/obp/ui/en/')
wait = WebDriverWait(navegador, 10)

filtro_div = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "v-select-optiongroup-xmltype")))
standards_input = filtro_div.find_element(By.XPATH, "//label[text()='Standards']/preceding-sibling::input")
standards_input.click()


search_box = navegador.find_element(By.XPATH, '//*[@id="obpui-105541713"]/div/div[2]/div/div/div[2]/div/div/div[2]/div/div/div/div/div/div[2]/div/div[2]/div/div[2]/div/div/div[2]/input')
search_box.send_keys('')
search_box.send_keys(Keys.RETURN)
wait

wait.until(EC.presence_of_element_located((By.CLASS_NAME, "v-label-std-title")))

def acha_titulos():
    titulos = {}
    while True:
        std_refs = navegador.find_elements(By.CLASS_NAME, "v-label-std-ref")
        std_titles = navegador.find_elements(By.CLASS_NAME, "v-label-std-title")
        for i in range(len(std_refs)):
            titulo = std_titles[i].text
            ref = std_refs[i].text
            titulos[ref] = titulo
        
        try:
            next_button = navegador.find_element(By.XPATH, "//div[@class='v-button v-widget i-paging v-button-i-paging last v-button-last']")
            if next_button.get_attribute("class") == "v-button v-widget i-paging v-button-i-paging last v-button-last v-disabled":
                break
            next_button.click()
            wait.until(EC.staleness_of(std_refs[-1]))
        except:
            break
            
    return titulos


titulos = acha_titulos()
print(titulos)
titulos.to_csv("titulos.csv", index=False)

procura os titulos e salva

In [ ]:
df = pd.DataFrame.from_dict(titulos, orient='index', columns=['descricao'])
df = df.explode()
df = df.reset_index()
df[['coluna1', 'coluna2', 'coluna3']] = df['index'].str.split(':', expand=True)
df[['coluna2', 'coluna3']] = df['coluna2'].str.split('(', expand=True)
df['coluna3'] = '(' + df['coluna3'] 
df = df[['coluna1', 'coluna2', 'coluna3', 'descricao']]
df = df.rename(columns={'coluna1': 'ISO', 'coluna2': 'VERSAO', 'coluna3': 'IDIOMA', 'descricao': 'DESCRICAO'})

print(df)
df.to_csv("iso.csv", index=False)

ValueError: Columns must be same length as key

In [16]:
df


,ISO,Título
